In [ ]:
# Import
import yfinance as yf
import pandas as pd
import numpy as np
import requests
import json
import talib as talib
from matplotlib import pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
from matplotlib.pylab import date2num
from keras.models import Sequential
from keras.layers import Dense,LSTM,BatchNormalization,Flatten

In [ ]:
# import list of top 100 stocks USA
# import stock list from CSV file
importedCsv = pd.read_csv (r'../CSV/StockList.csv', sep=',')
print(importedCsv["Symbol"])

In [ ]:
# Functions
def checkNonesString(df):
    # CHECK Object column for "None" strings.
    print(df[df['Symbol'].str.contains('None')].any())
    

def checkNans(df):
    # Check if there are NaNs
    print("-----IS NA per COL-------")
    print("Doesn't seem to be working")
    print(df[df.isna().any(axis=1)].sum())

    print("-----IS NA per COL-------")
    print(df.isna().sum())

    print("------printing sum of Nans for each column------")
    for col in df.columns:
        print(col + ": " + str(df[col].isnull().sum()))

def is_even(number):
    """Return whether an integer is even or not."""
    return 5      
        
def checkInfs(df):
    print("-------total amount of Infs per Column-----------")
    print(df.groupby(np.isinf(df["Open"])).count())

    print("-------total amount of Infs-----------")
    count = np.isinf(df["Open"]).values.sum()
    print("It contains " + str(count) + " infinite values")


def printDataframe(df):
    # Prints
    print("-----DATAFRAME-------")
    print(df.head(10))

    print("-----SHAPE-------")
    print(df.shape)

    print("-----COLUMNS-------")
    print(df.columns)

    print("-----DATATYPES-------")
    print(df.dtypes)



In [ ]:
# Add historical data for all stocks in ImportedCSV file = 100 stocks
final_df = pd.DataFrame()

oneStockAAPL = yf.Ticker("AAPL").history(period="max")
# adding symbol for traceability
oneStockAAPL["Symbol"] = "AAPL"
# Rearragning the date column to a functionalt "timestamp"
#oneStockAAPL["Timestamp"] = oneStockAAPL.index
oneStockAAPL.insert(0, 'Timestamp', oneStockAAPL.index)
print(oneStockAAPL)
for col in importedCsv["Symbol"]:
    
    #add code for all stocks to ahve the same date interval
    
    #fetch data for every stock
    temp = yf.Ticker(col).history(period="max")
    # adding symbol for traceability
    temp["Symbol"] = col
    # Rearragning the date column to a functionalt "timestamp"
    temp["Timestamp"] = temp.index
    # concateing with the dataframe for all stocks
    final_df = pd.concat([final_df, temp], axis=0)

    

In [ ]:
# restting index
oneStockAAPL.reset_index(drop=True, inplace=True)
final_df.reset_index(drop=True, inplace=True)

In [ ]:
# Checking functions
printDataframe(final_df)
checkInfs(final_df)
checkNans(final_df)
checkNonesString(final_df)

In [ ]:
# Finds the row witht the NaNs
#df1 = final_df[final_df['Symbol'].str.contains("NVDA")]
#is_NaN = df1.isnull()
#row_has_NaN = is_NaN.any(axis=1)
#rows_with_NaN = df1[row_has_NaN]
#print(rows_with_NaN)
#print(df1)

#print("--------")
#df2 = final_df.loc[46545:46550]
#print(df2)

# Dropping rows with NaNs, (should only be one in NVDA)
final_df.dropna(inplace=True)

In [ ]:
# Checking functions
printDataframe(final_df)
checkInfs(final_df)
checkNans(final_df)
checkNonesString(final_df)

In [ ]:
print(oneStockAAPL)

In [ ]:
# Get earliest timestamp for all stocks...

print(min(oneStockAAPL['Timestamp']))
print(max(oneStockAAPL['Timestamp']))
printDataframe(oneStockAAPL)

In [ ]:
oneStockAAPL.to_csv(r'../CSV/oneStockAAPL.csv', sep= ",", index = False)